<a href="https://colab.research.google.com/github/SAsandhiya/CODSOFT/blob/main/Basic_Network_Sniffer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask

In [ ]:
from flask import Flask, render_template_string, request, jsonify, send_file, session
import datetime
import random
import hashlib
import io
import csv
import json
import secrets
from functools import wraps

app = Flask(__name__)
app.secret_key = secrets.token_hex(32)

# In-memory storage (use database in production)
DEVICES = []
USERS = {}  # username: {password_hash, email, role}
ANALYTICS = {
    "total_devices": 0,
    "total_energy_saved": 0.0,
    "total_co2_avoided": 0.0,
    "total_ewaste_diverted": 0
}

# ---------- Authentication Decorator ----------
def login_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if 'user' not in session:
            return jsonify({"error": "Authentication required"}), 401
        return f(*args, **kwargs)
    return decorated_function

# ---------- HTML Template ----------
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <title>SecureWipeX – Enterprise Data Wiping & Device Recycling Platform</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet">
  <script src="https://cdnjs.cloudflare.com/ajax/libs/jspdf/2.5.1/jspdf.umd.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/qrcodejs/1.0.0/qrcode.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.0/dist/chart.umd.min.js"></script>

  <style>
    :root {
      --primary: #00d1c1;
      --secondary: #0aa3ff;
      --dark: #0b1220;
      --card: #0f1724;
      --card-hover: #1a2332;
      --muted: #9aa6b2;
      --success: #0f9d58;
      --warning: #f6c343;
      --danger: #e04646;
      --border: rgba(0,209,193,0.2);
    }

    * { margin: 0; padding: 0; box-sizing: border-box; }

    body {
      background: linear-gradient(135deg, var(--dark), #0f2a44);
      color: #e6eef6;
      font-family: 'Inter', 'Segoe UI', Arial, sans-serif;
      min-height: 10vh;
    }

    /* Navigation */
    .navbar {
      background: var(--card);
      border-bottom: 2px solid var(--border);
      padding: 1rem 0;
    }

    .navbar-brand {
      color: var(--primary) !important;
      font-size: 1.5rem;
      font-weight: 700;
    }

    .nav-link {
      color: var(--muted) !important;
      margin: 0 0.5rem;
      transition: color 0.3s;
    }

    .nav-link:hover, .nav-link.active {
      color: var(--primary) !important;
    }

    /* Hero Section */
    .hero {
      padding: 4rem 0;
      text-align: center;
      background: linear-gradient(135deg, rgba(0,209,193,0.1), rgba(10,163,255,0.1));
      border-radius: 20px;
      margin: 2rem 0;
    }

    .hero h1 {
      font-size: 3rem;
      font-weight: 700;
      background: linear-gradient(90deg, var(--primary), var(--secondary));
      -webkit-background-clip: text;
      -webkit-text-fill-color: transparent;
      margin-bottom: 1rem;
    }

    .hero p {
      font-size: 1.2rem;
      color: var(--muted);
      max-width: 700px;
      margin: 0 auto;
    }

    /* Cards */
    .container-box {
      background: var(--card);
      padding: 2rem;
      border-radius: 16px;
      margin-bottom: 2rem;
      box-shadow: 0 8px 24px rgba(0,0,0,0.4);
      border: 1px solid var(--border);
      transition: all 0.3s;
    }

    .container-box:hover {
      background: var(--card-hover);
      transform: translateY(-2px);
      box-shadow: 0 12px 32px rgba(0,0,0,0.5);
    }

    .glow {
      box-shadow: 0 0 24px rgba(0,209,193,0.15);
      border: 1px solid rgba(0,209,193,0.3);
    }

    /* Stats Cards */
    .stat-card {
      background: linear-gradient(135deg, var(--card), var(--card-hover));
      padding: 1.5rem;
      border-radius: 12px;
      text-align: center;
      border: 1px solid var(--border);
    }

    .stat-card h3 {
      font-size: 2.5rem;
      font-weight: 700;
      color: var(--primary);
      margin-bottom: 0.5rem;
    }

    .stat-card p {
      color: var(--muted);
      font-size: 0.9rem;
      text-transform: uppercase;
      letter-spacing: 1px;
    }

    /* Forms */
    .form-label {
      color: var(--muted);
      font-size: 0.9rem;
      font-weight: 500;
      margin-bottom: 0.5rem;
    }

    .form-control, .form-select {
      background: rgba(255,255,255,0.05);
      border: 1px solid var(--border);
      color: #e6eef6;
      padding: 0.75rem;
      border-radius: 8px;
      transition: all 0.3s;
    }

    .form-control:focus, .form-select:focus {
      background: rgba(255,255,255,0.08);
      border-color: var(--primary);
      color: #e6eef6;
      box-shadow: 0 0 0 3px rgba(0,209,193,0.1);
    }

    .form-control::placeholder {
      color: rgba(154,166,178,0.5);
    }

    /* Buttons */
    .btn-primary {
      background: linear-gradient(90deg, var(--primary), var(--secondary));
      border: none;
      padding: 0.75rem 2rem;
      font-weight: 600;
      border-radius: 8px;
      transition: all 0.3s;
    }

    .btn-primary:hover {
      transform: translateY(-2px);
      box-shadow: 0 8px 16px rgba(0,209,193,0.3);
    }

    .btn-outline-light {
      border: 1px solid var(--border);
      color: var(--primary);
      padding: 0.75rem 1.5rem;
      border-radius: 8px;
      transition: all 0.3s;
    }

    .btn-outline-light:hover {
      background: var(--primary);
      border-color: var(--primary);
      color: white;
    }

    /* Progress */
    .progress {
      background: rgba(255,255,255,0.05);
      border-radius: 10px;
      height: 24px;
      overflow: hidden;
    }

    .progress-bar {
      background: linear-gradient(90deg, var(--primary), var(--secondary));
      font-weight: 600;
      display: flex;
      align-items: center;
      justify-content: center;
    }

    /* Table */
    .table {
      color: #e6eef6;
    }

    .table th {
      color: var(--muted);
      font-size: 0.85rem;
      text-transform: uppercase;
      letter-spacing: 1px;
      border-bottom: 2px solid var(--border);
      padding: 1rem;
    }

    .table td {
      border-bottom: 1px solid rgba(154,166,178,0.1);
      padding: 1rem;
      vertical-align: middle;
    }

    .table tbody tr:hover {
      background: rgba(0,209,193,0.05);
    }

    /* Badges */
    .badge {
      padding: 0.5rem 1rem;
      border-radius: 6px;
      font-weight: 600;
      font-size: 0.85rem;
    }

    .badge-success { background: var(--success); }
    .badge-warning { background: var(--warning); color: var(--dark); }
    .badge-danger { background: var(--danger); }

    /* Log */
    #log {
      list-style: none;
      padding: 0;
      max-height: 300px;
      overflow-y: auto;
    }

    #log li {
      padding: 0.5rem 1rem;
      margin-bottom: 0.5rem;
      background: rgba(0,209,193,0.05);
      border-left: 3px solid var(--primary);
      border-radius: 4px;
      font-size: 0.9rem;
      animation: slideIn 0.3s ease;
    }

    @keyframes slideIn {
      from { opacity: 0; transform: translateX(-20px); }
      to { opacity: 1; transform: translateX(0); }
    }

    /* Risk indicators */
    .risk-low { color: var(--success); font-weight: 700; }
    .risk-medium { color: var(--warning); font-weight: 700; }
    .risk-high { color: var(--danger); font-weight: 700; }

    /* Charts */
    .chart-container {
      position: relative;
      height: 300px;
      margin-top: 1rem;
    }

    /* Modal */
    .modal-content {
      background: var(--card);
      color: #e6eef6;
      border: 1px solid var(--border);
      border-radius: 16px;
    }

    .modal-header {
      border-bottom: 1px solid var(--border);
    }

    .modal-footer {
      border-top: 1px solid var(--border);
    }

    /* Loading Animation */
    .loading {
      display: inline-block;
      width: 20px;
      height: 20px;
      border: 3px solid rgba(0,209,193,0.3);
      border-radius: 50%;
      border-top-color: var(--primary);
      animation: spin 1s linear infinite;
    }

    @keyframes spin {
      to { transform: rotate(360deg); }
    }

    /* Responsive */
    @media (max-width: 768px) {
      .hero h1 { font-size: 2rem; }
      .container-box { padding: 1.5rem; }
    }

    /* Login/Register Forms */
    .auth-container {
      max-width: 500px;
      margin: 4rem auto;
    }

    /* Page Sections */
    .page-section {
      display: none;
    }

    .page-section.active {
      display: block;
    }
  </style>
</head>
<body>
  <!-- Navigation -->
  <nav class="navbar navbar-expand-lg">
    <div class="container">
      <a class="navbar-brand" href="#" onclick="showSection('home')">
        <span style="font-size: 1.8rem;">🔒</span> SecureWipeX
      </a>
      <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarNav">
        <span class="navbar-toggler-icon"></span>
      </button>
      <div class="collapse navbar-collapse" id="navbarNav">
        <ul class="navbar-nav ms-auto">
          <li class="nav-item">
            <a class="nav-link" href="#" onclick="showSection('home')">Home</a>
          </li>
          <li class="nav-item">
            <a class="nav-link" href="#" onclick="showSection('wipe')">Wipe Device</a>
          </li>
          <li class="nav-item">
            <a class="nav-link" href="#" onclick="showSection('dashboard')">Dashboard</a>
          </li>
          <li class="nav-item">
            <a class="nav-link" href="#" onclick="showSection('analytics')">Analytics</a>
          </li>
          <li class="nav-item" id="authNav">
            <a class="nav-link" href="#" onclick="showSection('login')">Login</a>
          </li>
          <li class="nav-item" id="logoutNav" style="display:none;">
            <a class="nav-link" href="#" onclick="logout()">Logout (<span id="currentUser"></span>)</a>
          </li>
        </ul>
      </div>
    </div>
  </nav>

  <div class="container my-4">
    <!-- Home Section -->
    <div id="homeSection" class="page-section active">
      <div class="hero">
        <h1>Enterprise Data Wiping Platform</h1>
        <p>Secure, compliant, and verifiable data destruction with comprehensive audit trails and environmental impact tracking</p>
        <div class="mt-4">
          <button class="btn btn-primary btn-lg" onclick="showSection('wipe')">Start Wiping Now</button>
          <button class="btn btn-outline-light btn-lg ms-2" onclick="showSection('analytics')">View Analytics</button>
        </div>
      </div>

      <!-- Global Statistics -->
      <div class="row mt-5">
        <div class="col-md-3 mb-3">
          <div class="stat-card">
            <h3 id="globalDevices">0</h3>
            <p>Devices Wiped</p>
          </div>
        </div>
        <div class="col-md-3 mb-3">
          <div class="stat-card">
            <h3 id="globalEnergy">0</h3>
            <p>kWh Saved</p>
          </div>
        </div>
        <div class="col-md-3 mb-3">
          <div class="stat-card">
            <h3 id="globalCO2">0</h3>
            <p>kg CO₂ Avoided</p>
          </div>
        </div>
        <div class="col-md-3 mb-3">
          <div class="stat-card">
            <h3 id="globalEwaste">0</h3>
            <p>E-waste Diverted</p>
          </div>
        </div>
      </div>

      <!-- Features -->
      <div class="row mt-4">
        <div class="col-md-4 mb-3">
          <div class="container-box">
            <h4 style="color: var(--primary);">🔐 Military-Grade Security</h4>
            <p class="text-muted">Multiple secure erase algorithms including NIST, DoD, and custom ML-enhanced protocols</p>
          </div>
        </div>
        <div class="col-md-4 mb-3">
          <div class="container-box">
            <h4 style="color: var(--primary);">📜 Compliance Ready</h4>
            <p class="text-muted">GDPR, HIPAA, ISO 27001, and DoD compliant with verifiable certificates</p>
          </div>
        </div>
        <div class="col-md-4 mb-3">
          <div class="container-box">
            <h4 style="color: var(--primary);">🌍 Environmental Impact</h4>
            <p class="text-muted">Track energy savings, CO₂ reduction, and e-waste diversion in real-time</p>
          </div>
        </div>
      </div>
    </div>

    <!-- Login Section -->
    <div id="loginSection" class="page-section">
      <div class="auth-container">
        <div class="container-box">
          <h2 class="text-center mb-4">Login to SecureWipeX</h2>
          <form onsubmit="login(event)">
            <div class="mb-3">
              <label class="form-label">Username</label>
              <input type="text" class="form-control" id="loginUsername" required>
            </div>
            <div class="mb-3">
              <label class="form-label">Password</label>
              <input type="password" class="form-control" id="loginPassword" required>
            </div>
            <button type="submit" class="btn btn-primary w-100">Login</button>
          </form>
          <p class="text-center mt-3 text-muted">
            Don't have an account? <a href="#" onclick="showSection('register')" style="color: var(--primary);">Register</a>
          </p>
        </div>
      </div>
    </div>

    <!-- Register Section -->
    <div id="registerSection" class="page-section">
      <div class="auth-container">
        <div class="container-box">
          <h2 class="text-center mb-4">Create Account</h2>
          <form onsubmit="register(event)">
            <div class="mb-3">
              <label class="form-label">Username</label>
              <input type="text" class="form-control" id="regUsername" required>
            </div>
            <div class="mb-3">
              <label class="form-label">Email</label>
              <input type="email" class="form-control" id="regEmail" required>
            </div>
            <div class="mb-3">
              <label class="form-label">Password</label>
              <input type="password" class="form-control" id="regPassword" required minlength="6">
            </div>
            <button type="submit" class="btn btn-primary w-100">Register</button>
          </form>
          <p class="text-center mt-3 text-muted">
            Already have an account? <a href="#" onclick="showSection('login')" style="color: var(--primary);">Login</a>
          </p>
        </div>
      </div>
    </div>

    <!-- Wipe Device Section -->
    <div id="wipeSection" class="page-section">
      <h2 class="mb-4">Secure Device Wipe</h2>

      <div class="container-box glow">
        <h5 class="mb-3">Device Information</h5>
        <div class="row g-3">
          <div class="col-md-4">
            <label class="form-label">Organization / Department</label>
            <input id="org" class="form-control" placeholder="e.g., IT Department">
          </div>
          <div class="col-md-4">
            <label class="form-label">Owner Name</label>
            <input id="owner" class="form-control" placeholder="e.g., John Doe" required>
          </div>
          <div class="col-md-4">
            <label class="form-label">Device Type</label>
            <select id="device" class="form-select">
              <option value="Laptop">Laptop</option>
              <option value="Desktop">Desktop</option>
              <option value="Phone">Phone</option>
              <option value="Tablet">Tablet</option>
              <option value="Hard Drive">Hard Drive</option>
              <option value="SSD">SSD</option>
              <option value="USB Drive">USB Drive</option>
            </select>
          </div>
          <div class="col-md-4">
            <label class="form-label">Manufacturer</label>
            <input id="manufacturer" class="form-control" placeholder="e.g., Dell, Apple, Samsung">
          </div>
          <div class="col-md-4">
            <label class="form-label">Model</label>
            <input id="model" class="form-control" placeholder="e.g., XPS 15">
          </div>
          <div class="col-md-4">
            <label class="form-label">Serial Number</label>
            <input id="serial" class="form-control" placeholder="Serial Number" required>
          </div>
          <div class="col-md-4">
            <label class="form-label">Storage Capacity (GB)</label>
            <input id="storage" class="form-control" type="number" min="1" placeholder="512" required>
          </div>
          <div class="col-md-4">
            <label class="form-label">Wipe Algorithm</label>
            <select id="algorithm" class="form-select">
              <option value="nist">NIST SP 800-88 (3 passes) - Recommended</option>
              <option value="dod">DoD 5220.22-M (7 passes) - High Security</option>
              <option value="litegbm">LiteGBM SecureErase (3 passes) - Fast</option>
              <option value="gbm">GBM SecureErase (5 passes) - Balanced</option>
              <option value="randomx">RandomX Secure (9 passes) - Maximum</option>
              <option value="gutmann">Gutmann (35 passes) - Ultimate</option>
            </select>
          </div>
          <div class="col-md-4">
            <label class="form-label">Priority</label>
            <select id="priority" class="form-select">
              <option value="normal">Normal</option>
              <option value="high">High</option>
              <option value="urgent">Urgent</option>
            </select>
          </div>
          <div class="col-12">
            <label class="form-label">Additional Notes (Optional)</label>
            <textarea id="notes" class="form-control" rows="2" placeholder="Any additional information..."></textarea>
          </div>
          <div class="col-12">
            <button id="startBtn" class="btn btn-primary btn-lg" onclick="startWipe()">
              🔒 Start Secure Wipe
            </button>
          </div>
        </div>
      </div>

      <!-- Progress Section -->
      <div id="progressContainer" class="container-box mt-4" style="display:none;">
        <div class="row">
          <div class="col-md-8">
            <h5>Wiping Progress <small class="text-muted" id="progressAlgo"></small></h5>
            <div class="progress mb-3">
              <div id="progressBar" class="progress-bar progress-bar-striped progress-bar-animated" role="progressbar" style="width:0%">0%</div>
            </div>
            <ul id="log"></ul>
          </div>
          <div class="col-md-4">
            <h6 class="text-muted mb-3">Live Impact Metrics</h6>
            <div class="stat-card mb-3">
              <h3 id="energy" style="font-size: 1.8rem;">0 kWh</h3>
              <p style="margin: 0;">Energy Used</p>
            </div>
            <div class="stat-card mb-3">
              <h3 id="co2" style="font-size: 1.8rem;">0 g</h3>
              <p style="margin: 0;">CO₂ Avoided</p>
            </div>
            <div class="stat-card">
              <h3 id="ewaste" style="font-size: 1.8rem;">1</h3>
              <p style="margin: 0;">Devices Processed</p>
            </div>
          </div>
        </div>
      </div>

      <!-- Certificate Display -->
      <div id="certificateBox" class="container-box" style="display:none;">
        <h5>Wipe Certificate Generated</h5>
        <pre id="certificateText" style="background: rgba(0,0,0,0.3); padding: 1rem; border-radius: 8px; white-space: pre-wrap;"></pre>
        <div class="mt-3">
          <button class="btn btn-success" onclick="downloadCertificate()">📄 Download PDF Certificate</button>
          <button class="btn btn-outline-light" onclick="showSection('dashboard')">View Dashboard</button>
        </div>
      </div>
    </div>

    <!-- Dashboard Section -->
    <div id="dashboardSection" class="page-section">
      <div class="d-flex justify-content-between align-items-center mb-4">
        <h2>Device Dashboard</h2>
        <button class="btn btn-outline-light" onclick="exportCSV()">📊 Export CSV</button>
      </div>

      <div class="container-box">
        <div class="table-responsive">
          <table class="table">
            <thead>
              <tr>
                <th>Timestamp</th>
                <th>Organization</th>
                <th>Owner</th>
                <th>Device</th>
                <th>Model</th>
                <th>Serial</th>
                <th>Storage</th>
                <th>Algorithm</th>
                <th>Status</th>
                <th>Certificate</th>
              </tr>
            </thead>
            <tbody id="dashboardBody">
              <tr>
                <td colspan="10" class="text-center text-muted">No devices wiped yet</td>
              </tr>
            </tbody>
          </table>
        </div>
      </div>
    </div>

    <!-- Analytics Section -->
    <div id="analyticsSection" class="page-section">
      <h2 class="mb-4">Analytics & Reports</h2>

      <div class="row mb-4">
        <div class="col-md-3">
          <div class="stat-card">
            <h3 id="analyticsDevices">0</h3>
            <p>Total Devices</p>
          </div>
        </div>
        <div class="col-md-3">
          <div class="stat-card">
            <h3 id="analyticsEnergy">0 kWh</h3>
            <p>Energy Consumption</p>
          </div>
        </div>
        <div class="col-md-3">
          <div class="stat-card">
            <h3 id="analyticsCO2">0 kg</h3>
            <p>CO₂ Avoided</p>
          </div>
        </div>
        <div class="col-md-3">
          <div class="stat-card">
            <h3 id="analyticsCompliance">100%</h3>
            <p>Compliance Rate</p>
          </div>
        </div>
      </div>

      <div class="row">
        <div class="col-md-6 mb-4">
          <div class="container-box">
            <h5>Devices by Type</h5>
            <div class="chart-container">
              <canvas id="deviceTypeChart"></canvas>
            </div>
          </div>
        </div>
        <div class="col-md-6 mb-4">
          <div class="container-box">
            <h5>Algorithms Used</h5>
            <div class="chart-container">
              <canvas id="algorithmChart"></canvas>
            </div>
          </div>
        </div>
      </div>

      <div class="container-box">
        <h5>Environmental Impact Over Time</h5>
        <div class="chart-container" style="height: 400px;">
          <canvas id="impactChart"></canvas>
        </div>
      </div>
    </div>
  </div>

  <!-- Certificate Modal -->
  <div class="modal fade" id="certificateModal" tabindex="-1">
    <div class="modal-dialog modal-lg modal-dialog-centered">
      <div class="modal-content">
        <div class="modal-header">
          <h5 class="modal-title">SecureWipeX Certificate</h5>
          <button type="button" class="btn-close" data-bs-dismiss="modal"></button>
        </div>
        <div class="modal-body">
          <div class="row">
            <div class="col-md-8">
              <div id="certificateContent"></div>
            </div>
            <div class="col-md-4">
              <h6 class="text-muted mb-3">Verification</h6>
              <div id="qrcode" class="mb-3"></div>
              <p><strong>SHA256:</strong><br><span id="certHash" style="font-size: 0.8rem; word-break: break-all;"></span></p>
              <p><strong>Blockchain TX:</strong><br><span id="txid" style="font-size: 0.8rem; word-break: break-all;"></span></p>
              <p><strong>Risk Score:</strong> <span id="riskScore"></span></p>
              <div id="complianceBadges" class="mt-3"></div>
            </div>
          </div>
        </div>
        <div class="modal-footer">
          <button class="btn btn-success" onclick="downloadCertificate()">Download PDF</button>
          <button class="btn btn-secondary" data-bs-dismiss="modal">Close</button>
        </div>
      </div>
    </div>
  </div>

  <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/js/bootstrap.bundle.min.js"></script>

  <script>
    // Global state
    let currentUser = null;
    let deviceData = {};
    let latestCert = null;
    let allDevices = [];
    let charts = {};

    // Algorithm metadata
    const ALGO_META = {
      "nist": { name: "NIST SP 800-88", passes: 3, speedFactor: 1.0, compliance: ["GDPR", "ISO 27001"] },
      "dod": { name: "DoD 5220.22-M", passes: 7, speedFactor: 0.6, compliance: ["DoD", "ISO 27001"] },
      "litegbm": { name: "LiteGBM SecureErase", passes: 3, speedFactor: 1.6, compliance: ["GDPR", "HIPAA"] },
      "gbm": { name: "GBM SecureErase", passes: 5, speedFactor: 1.1, compliance: ["GDPR", "ISO 27001", "HIPAA"] },
      "randomx": { name: "RandomX Secure", passes: 9, speedFactor: 0.5, compliance: ["DoD", "ISO 27001"] },
      "gutmann": { name: "Gutmann Method", passes: 35, speedFactor: 0.3, compliance: ["DoD", "ISO 27001", "NSA"] }
    };

    // Initialize app
    document.addEventListener('DOMContentLoaded', function() {
      checkAuth();
      loadGlobalStats();
      loadDashboard();
    });

    // Authentication
    function checkAuth() {
      fetch('/check_auth')
        .then(r => r.json())
        .then(data => {
          if (data.authenticated) {
            currentUser = data.username;
            document.getElementById('authNav').style.display = 'none';
            document.getElementById('logoutNav').style.display = 'block';
            document.getElementById('currentUser').textContent = currentUser;
          }
        });
    }

    function login(event) {
      event.preventDefault();
      const username = document.getElementById('loginUsername').value;
      const password = document.getElementById('loginPassword').value;

      fetch('/login', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ username, password })
      })
      .then(r => r.json())
      .then(data => {
        if (data.success) {
          currentUser = username;
          document.getElementById('authNav').style.display = 'none';
          document.getElementById('logoutNav').style.display = 'block';
          document.getElementById('currentUser').textContent = currentUser;
          showSection('home');
          alert('Login successful!');
        } else {
          alert(data.error || 'Login failed');
        }
      });
    }

    function register(event) {
      event.preventDefault();
      const username = document.getElementById('regUsername').value;
      const email = document.getElementById('regEmail').value;
      const password = document.getElementById('regPassword').value;

      fetch('/register', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ username, email, password })
      })
      .then(r => r.json())
      .then(data => {
        if (data.success) {
          alert('Registration successful! Please login.');
          showSection('login');
        } else {
          alert(data.error || 'Registration failed');
        }
      });
    }

    function logout() {
      fetch('/logout', { method: 'POST' })
        .then(() => {
          currentUser = null;
          document.getElementById('authNav').style.display = 'block';
          document.getElementById('logoutNav').style.display = 'none';
          showSection('home');
          alert('Logged out successfully');
        });
    }

    // Navigation
    function showSection(section) {
      document.querySelectorAll('.page-section').forEach(s => s.classList.remove('active'));
      document.getElementById(section + 'Section').classList.add('active');

      document.querySelectorAll('.nav-link').forEach(l => l.classList.remove('active'));

      if (section === 'analytics') {
        loadAnalytics();
      }
    }

    // Load global statistics
    function loadGlobalStats() {
      fetch('/analytics')
        .then(r => r.json())
        .then(data => {
          document.getElementById('globalDevices').textContent = data.total_devices;
          document.getElementById('globalEnergy').textContent = data.total_energy_saved.toFixed(2);
          document.getElementById('globalCO2').textContent = (data.total_co2_avoided / 1000).toFixed(2);
          document.getElementById('globalEwaste').textContent = data.total_ewaste_diverted;
        });
    }

    // Start wipe process
    function startWipe() {
      if (!currentUser) {
        alert('Please login to wipe devices');
        showSection('login');
        return;
      }

      deviceData = {
        org: document.getElementById('org').value || 'Individual',
        owner: document.getElementById('owner').value,
        device: document.getElementById('device').value,
        manufacturer: document.getElementById('manufacturer').value || 'Unknown',
        model: document.getElementById('model').value || 'Unknown',
        serial: document.getElementById('serial').value,
        storage: parseFloat(document.getElementById('storage').value),
        algorithmKey: document.getElementById('algorithm').value,
        priority: document.getElementById('priority').value,
        notes: document.getElementById('notes').value
      };

      if (!deviceData.owner || !deviceData.serial || !deviceData.storage) {
        alert('Please fill in all required fields');
        return;
      }

      document.getElementById('progressContainer').style.display = 'block';
      document.getElementById('certificateBox').style.display = 'none';
      document.getElementById('log').innerHTML = '';
      document.getElementById('startBtn').disabled = true;

      const meta = ALGO_META[deviceData.algorithmKey];
      document.getElementById('progressAlgo').textContent = meta.name + ' • ' + meta.passes + ' passes';

      const baseSteps = meta.passes * 4;
      const totalSteps = Math.max(8, Math.round(baseSteps * (256 / deviceData.storage) * (1 / meta.speedFactor)));
      let step = 0;
      const progressBar = document.getElementById('progressBar');
      const energyBasePerPass = 0.03;
      const energyEstimated = meta.passes * energyBasePerPass * (deviceData.storage / 256);
      const co2PerKwh = 475;
      let energySoFar = 0;

      const intervalMs = Math.max(150, Math.round(600 / meta.speedFactor));

      const messages = [
        'Initializing secure wipe protocol...',
        'Verifying device connection...',
        'Allocating secure overwrite buffers...',
        'Starting first pass overwrite...',
        'Overwriting logical sectors...',
        'Verifying overwritten data...',
        'Erasing hidden areas (HPA/DCO)...',
        'Sanitizing SSD blocks (TRIM)...',
        'Performing cryptographic erase...',
        'Running verification pass...',
        'Finalizing secure wipe...',
        'Generating certificate hash...',
        'Creating blockchain verification...',
        'Wipe completed successfully!'
      ];

      const interval = setInterval(() => {
        step++;
        const pct = Math.round((step / totalSteps) * 100);
        progressBar.style.width = pct + '%';
        progressBar.textContent = pct + '%';

        const msgIndex = Math.min(messages.length - 1, Math.floor((step / totalSteps) * messages.length));
        const li = document.createElement('li');
        li.textContent = messages[msgIndex];
        document.getElementById('log').appendChild(li);
        document.getElementById('log').scrollTop = document.getElementById('log').scrollHeight;

        energySoFar = (step / totalSteps) * energyEstimated;
        document.getElementById('energy').textContent = energySoFar.toFixed(3) + ' kWh';
        const co2 = energySoFar * co2PerKwh;
        document.getElementById('co2').textContent = co2.toFixed(1) + ' g';

        if (step >= totalSteps) {
          clearInterval(interval);
          progressBar.style.width = '100%';
          progressBar.textContent = '100%';
          finalizeWipe(totalSteps, intervalMs);
        }
      }, intervalMs);
    }

    // Finalize wipe and get certificate
    function finalizeWipe(totalSteps, intervalMs) {
      deviceData.duration_ms = totalSteps * intervalMs;

      fetch('/wipe_device', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify(deviceData)
      })
      .then(r => r.json())
      .then(cert => {
        latestCert = cert;
        showCertificate(cert);
        loadDashboard();
        loadGlobalStats();
        document.getElementById('startBtn').disabled = false;
      })
      .catch(err => {
        console.error('Certificate error', err);
        alert('Failed to generate certificate');
        document.getElementById('startBtn').disabled = false;
      });
    }

    // Display certificate
    function showCertificate(cert) {
      const text = `SecureWipeX Certificate
Certificate ID: ${cert.certificate_id}
Organization: ${cert.org}
Owner: ${cert.owner}
Device: ${cert.device}
Manufacturer: ${cert.manufacturer}
Model: ${cert.model}
Serial: ${cert.serial}
Storage: ${cert.storage}
Algorithm: ${cert.algorithm}
Status: ${cert.status}
Risk Score: ${cert.risk_score} (${cert.risk_label})
Energy Used: ${cert.energy_kwh} kWh
CO₂ Avoided: ${cert.co2_g} g
Issued: ${cert.timestamp}`;

      document.getElementById('certificateText').textContent = text;
      document.getElementById('certificateContent').innerHTML = text.replace(/\n/g, '<br/>');
      document.getElementById('certHash').textContent = cert.sha256;
      document.getElementById('txid').textContent = cert.txid;

      const riskClass = cert.risk_label === 'LOW' ? 'risk-low' : cert.risk_label === 'MEDIUM' ? 'risk-medium' : 'risk-high';
      document.getElementById('riskScore').innerHTML = `<span class="${riskClass}">${cert.risk_score} (${cert.risk_label})</span>`;

      const badges = cert.compliance.map(c => `<span class="badge badge-success">${c}</span>`).join(' ');
      document.getElementById('complianceBadges').innerHTML = badges;

      document.getElementById('qrcode').innerHTML = '';
      new QRCode(document.getElementById('qrcode'), {
        text: cert.verify_url || cert.sha256,
        width: 140,
        height: 140
      });

      document.getElementById('certificateBox').style.display = 'block';

      const certModal = new bootstrap.Modal(document.getElementById('certificateModal'));
      certModal.show();
    }

    // Load dashboard
    function loadDashboard() {
      fetch('/devices')
        .then(r => r.json())
        .then(devices => {
          allDevices = devices;
          const tbody = document.getElementById('dashboardBody');

          if (devices.length === 0) {
            tbody.innerHTML = '<tr><td colspan="10" class="text-center text-muted">No devices wiped yet</td></tr>';
            return;
          }

          tbody.innerHTML = '';
          devices.reverse().forEach(cert => {
            const tr = document.createElement('tr');
            const statusBadge = cert.status === 'Wiped Successfully' ?
              '<span class="badge badge-success">Success</span>' :
              '<span class="badge badge-warning">Pending</span>';

            tr.innerHTML = `
              <td>${cert.timestamp}</td>
              <td>${escapeHtml(cert.org)}</td>
              <td>${escapeHtml(cert.owner)}</td>
              <td>${escapeHtml(cert.device)}</td>
              <td>${escapeHtml(cert.model)}</td>
              <td>${escapeHtml(cert.serial)}</td>
              <td>${cert.storage}</td>
              <td>${cert.algorithm}</td>
              <td>${statusBadge}</td>
              <td><button class="btn btn-sm btn-outline-light" onclick='viewCertificate(${JSON.stringify(cert).replace(/'/g, "&apos;")})'>View</button></td>
            `;
            tbody.appendChild(tr);
          });
        });
    }

    // View certificate
    function viewCertificate(cert) {
      latestCert = cert;
      showCertificate(cert);
    }

    // Download PDF certificate
    function downloadCertificate() {
      if (!latestCert) {
        alert('No certificate available');
        return;
      }

      const { jsPDF } = window.jspdf;
      const doc = new jsPDF();

      doc.setFontSize(20);
      doc.setTextColor(0, 209, 193);
      doc.text('SecureWipeX Certificate', 20, 20);

      doc.setFontSize(10);
      doc.setTextColor(100, 100, 100);
      doc.text('Secure Data Wiping & Device Recycling', 20, 28);

      doc.setDrawColor(0, 209, 193);
      doc.line(20, 32, 190, 32);

      doc.setFontSize(11);
      doc.setTextColor(0, 0, 0);
      let y = 45;

      const fields = [
        ['Certificate ID', latestCert.certificate_id],
        ['Organization', latestCert.org],
        ['Owner', latestCert.owner],
        ['Device Type', latestCert.device],
        ['Manufacturer', latestCert.manufacturer],
        ['Model', latestCert.model],
        ['Serial Number', latestCert.serial],
        ['Storage Capacity', latestCert.storage],
        ['Wipe Algorithm', latestCert.algorithm],
        ['Status', latestCert.status],
        ['Risk Score', `${latestCert.risk_score} (${latestCert.risk_label})`],
        ['Energy Used', `${latestCert.energy_kwh} kWh`],
        ['CO₂ Avoided', `${latestCert.co2_g} g`],
        ['Timestamp', latestCert.timestamp]
      ];

      fields.forEach(([label, value]) => {
        doc.setFont(undefined, 'bold');
        doc.text(label + ':', 20, y);
        doc.setFont(undefined, 'normal');
        doc.text(String(value), 80, y);
        y += 8;
      });

      y += 5;
      doc.setFontSize(10);
      doc.setFont(undefined, 'bold');
      doc.text('Verification:', 20, y);
      y += 7;
      doc.setFont(undefined, 'normal');
      doc.setFontSize(8);
      doc.text('SHA256: ' + latestCert.sha256, 20, y);
      y += 5;
      doc.text('Blockchain TX: ' + latestCert.txid, 20, y);
      y += 10;

      doc.setFontSize(9);
      doc.text('Compliance: ' + latestCert.compliance.join(', '), 20, y);

      y += 15;
      doc.setFontSize(8);
      doc.setTextColor(150, 150, 150);
      doc.text('This certificate verifies that the device has been securely wiped according to industry standards.', 20, y);
      y += 5;
      doc.text('For verification, scan the QR code or visit: securewipex.com/verify', 20, y);

      doc.save(`SecureWipeX_Certificate_${latestCert.certificate_id}.pdf`);
    }

    // Export CSV
    function exportCSV() {
      window.open('/devices_csv', '_blank');
    }

    // Load analytics
    function loadAnalytics() {
      fetch('/analytics')
        .then(r => r.json())
        .then(data => {
          document.getElementById('analyticsDevices').textContent = data.total_devices;
          document.getElementById('analyticsEnergy').textContent = data.total_energy_saved.toFixed(2) + ' kWh';
          document.getElementById('analyticsCO2').textContent = (data.total_co2_avoided / 1000).toFixed(2) + ' kg';
          document.getElementById('analyticsCompliance').textContent = '100%';

          createCharts(data);
        });
    }

    // Create charts
    function createCharts(data) {
      const deviceTypes = {};
      const algorithms = {};

      allDevices.forEach(device => {
        deviceTypes[device.device] = (deviceTypes[device.device] || 0) + 1;
        algorithms[device.algorithm] = (algorithms[device.algorithm] || 0) + 1;
      });

      if (charts.deviceType) charts.deviceType.destroy();
      if (charts.algorithm) charts.algorithm.destroy();
      if (charts.impact) charts.impact.destroy();

      const ctx1 = document.getElementById('deviceTypeChart').getContext('2d');
      charts.deviceType = new Chart(ctx1, {
        type: 'doughnut',
        data: {
          labels: Object.keys(deviceTypes),
          datasets: [{
            data: Object.values(deviceTypes),
            backgroundColor: ['#00d1c1', '#0aa3ff', '#f6c343', '#e04646', '#0f9d58']
          }]
        },
        options: {
          responsive: true,
          maintainAspectRatio: false,
          plugins: {
            legend: { labels: { color: '#e6eef6' } }
          }
        }
      });

      const ctx2 = document.getElementById('algorithmChart').getContext('2d');
      charts.algorithm = new Chart(ctx2, {
        type: 'bar',
        data: {
          labels: Object.keys(algorithms),
          datasets: [{
            label: 'Uses',
            data: Object.values(algorithms),
            backgroundColor: '#00d1c1'
          }]
        },
        options: {
          responsive: true,
          maintainAspectRatio: false,
          plugins: {
            legend: { labels: { color: '#e6eef6' } }
          },
          scales: {
            y: { ticks: { color: '#e6eef6' }, grid: { color: 'rgba(154,166,178,0.1)' } },
            x: { ticks: { color: '#e6eef6' }, grid: { color: 'rgba(154,166,178,0.1)' } }
          }
        }
      });

      const impactData = allDevices.map((d, i) => ({
        x: i + 1,
        y: parseFloat(d.co2_g) / 1000
      }));

      const ctx3 = document.getElementById('impactChart').getContext('2d');
      charts.impact = new Chart(ctx3, {
        type: 'line',
        data: {
          datasets: [{
            label: 'CO₂ Avoided (kg)',
            data: impactData,
            borderColor: '#00d1c1',
            backgroundColor: 'rgba(0,209,193,0.1)',
            fill: true
          }]
        },
        options: {
          responsive: true,
          maintainAspectRatio: false,
          plugins: {
            legend: { labels: { color: '#e6eef6' } }
          },
          scales: {
            y: { ticks: { color: '#e6eef6' }, grid: { color: 'rgba(154,166,178,0.1)' } },
            x: { ticks: { color: '#e6eef6' }, grid: { color: 'rgba(154,166,178,0.1)' } }
          }
        }
      });
    }

    function escapeHtml(text) {
      const div = document.createElement('div');
      div.textContent = text;
      return div.innerHTML;
    }
  </script>
</body>
</html>
"""

# ---------- Backend Routes ----------

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

def compute_sha256(text):
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

def simulate_risk_score(device_storage_gb, passes, algorithm_key):
    base = 60 + (device_storage_gb / 512) * 20
    reduction = passes * 8 + random.uniform(-8, 8)
    score = max(1, min(99, int(base - reduction)))

    if score >= 66:
        label = "LOW"
    elif score >= 40:
        label = "MEDIUM"
    else:
        label = "HIGH"

    return score, label

def compute_energy_co2(storage_gb, passes):
    energy_per_pass_kwh = 0.03 * (storage_gb / 256)
    total_kwh = round(energy_per_pass_kwh * passes, 3)
    co2_g = int(total_kwh * 475)
    return total_kwh, co2_g

@app.route("/")
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route("/register", methods=["POST"])
def register():
    data = request.get_json() or {}
    username = data.get("username", "").strip()
    email = data.get("email", "").strip()
    password = data.get("password", "")

    if not username or not email or not password:
        return jsonify({"success": False, "error": "All fields required"}), 400

    if username in USERS:
        return jsonify({"success": False, "error": "Username already exists"}), 400

    USERS[username] = {
        "password_hash": hash_password(password),
        "email": email,
        "role": "user",
        "created_at": datetime.datetime.now().isoformat()
    }

    return jsonify({"success": True, "message": "Registration successful"})

@app.route("/login", methods=["POST"])
def login():
    data = request.get_json() or {}
    username = data.get("username", "").strip()
    password = data.get("password", "")

    if username not in USERS:
        return jsonify({"success": False, "error": "Invalid credentials"}), 401

    if USERS[username]["password_hash"] != hash_password(password):
        return jsonify({"success": False, "error": "Invalid credentials"}), 401

    session["user"] = username
    return jsonify({"success": True, "username": username})

@app.route("/logout", methods=["POST"])
def logout():
    session.pop("user", None)
    return jsonify({"success": True})

@app.route("/check_auth")
def check_auth():
    if "user" in session:
        return jsonify({"authenticated": True, "username": session["user"]})
    return jsonify({"authenticated": False})

@app.route("/wipe_device", methods=["POST"])
@login_required
def wipe_device():
    data = request.get_json() or {}

    algo_map = {
        "nist": ("NIST SP 800-88", 3, ["GDPR", "ISO 27001"]),
        "dod": ("DoD 5220.22-M", 7, ["DoD", "ISO 27001"]),
        "litegbm": ("LiteGBM SecureErase", 3, ["GDPR", "HIPAA"]),
        "gbm": ("GBM SecureErase", 5, ["GDPR", "ISO 27001", "HIPAA"]),
        "randomx": ("RandomX Secure", 9, ["DoD", "ISO 27001"]),
        "gutmann": ("Gutmann Method", 35, ["DoD", "ISO 27001", "NSA"])
    }

    key = data.get("algorithmKey", "nist")
    algorithm_name, passes, compliance = algo_map.get(key, algo_map["nist"])

    storage_val = float(data.get("storage", 128))
    cert_id = "SWX-" + str(random.randint(100000, 999999))
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    base_for_hash = f"{cert_id}|{data.get('serial', '')}|{timestamp}"
    sha = compute_sha256(base_for_hash)
    txid = hashlib.sha256((sha + str(random.random())).encode()).hexdigest()

    risk_score, risk_label = simulate_risk_score(storage_val, passes, key)
    energy_kwh, co2_g = compute_energy_co2(storage_val, passes)

    speed_rank_val = int(50 + random.uniform(-18, 28))
    speed_rank_str = f"{speed_rank_val}th percentile"

    certificate = {
        "certificate_id": cert_id,
        "org": data.get("org", "Individual"),
        "owner": data.get("owner", "Unknown"),
        "device": data.get("device", "Unknown"),
        "manufacturer": data.get("manufacturer", "Unknown"),
        "model": data.get("model", "Unknown"),
        "serial": data.get("serial", "Unknown"),
        "storage": f"{int(storage_val)} GB",
        "algorithm": algorithm_name,
        "status": "Wiped Successfully",
        "timestamp": timestamp,
        "sha256": sha,
        "txid": txid,
        "risk_score": risk_score,
        "risk_label": risk_label,
        "energy_kwh": energy_kwh,
        "co2_g": co2_g,
        "ewaste_diverted": 1,
        "compliance": compliance,
        "speed_rank": speed_rank_str,
        "verify_url": f"https://securewipex.com/verify/{sha}",
        "priority": data.get("priority", "normal"),
        "notes": data.get("notes", ""),
        "user": session.get("user")
    }

    DEVICES.append(certificate)

    ANALYTICS["total_devices"] += 1
    ANALYTICS["total_energy_saved"] += energy_kwh
    ANALYTICS["total_co2_avoided"] += co2_g
    ANALYTICS["total_ewaste_diverted"] += 1

    return jsonify(certificate)

@app.route("/devices")
def get_devices():
    return jsonify(DEVICES)

@app.route("/analytics")
def get_analytics():
    return jsonify(ANALYTICS)

@app.route("/devices_csv")
def devices_csv():
    si = io.StringIO()
    cw = csv.writer(si)

    headers = [
        "certificate_id", "timestamp", "user", "org", "owner", "device",
        "manufacturer", "model", "serial", "storage", "algorithm", "status",
        "sha256", "txid", "risk_score", "risk_label", "energy_kwh", "co2_g",
        "compliance", "priority", "notes"
    ]
    cw.writerow(headers)

    for d in DEVICES:
        cw.writerow([
            d.get("certificate_id", ""),
            d.get("timestamp", ""),
            d.get("user", ""),
            d.get("org", ""),
            d.get("owner", ""),
            d.get("device", ""),
            d.get("manufacturer", ""),
            d.get("model", ""),
            d.get("serial", ""),
            d.get("storage", ""),
            d.get("algorithm", ""),
            d.get("status", ""),
            d.get("sha256", ""),
            d.get("txid", ""),
            d.get("risk_score", ""),
            d.get("risk_label", ""),
            d.get("energy_kwh", ""),
            d.get("co2_g", ""),
            ', '.join(d.get("compliance", [])),
            d.get("priority", ""),
            d.get("notes", "")
        ])

    mem = io.BytesIO()
    mem.write(si.getvalue().encode('utf-8'))
    mem.seek(0)
    si.close()

    return send_file(
        mem,
        mimetype="text/csv",
        download_name=f"securewipex_devices_{datetime.datetime.now().strftime('%Y%m%d')}.csv",
        as_attachment=True
    )

if __name__ == "__main__":
    print("=" * 60)
    print("SecureWipeX - Enterprise Data Wiping Platform")
    print("=" * 60)
    print("Starting server at http://127.0.0.1:5000")
    print("Features:")
    print("  • User Authentication (Register/Login)")
    print("  • Multiple Wipe Algorithms (NIST, DoD, GBM, etc.)")
    print("  • Certificate Generation with QR Codes")
    print("  • Real-time Analytics & Charts")
    print("  • CSV Export Functionality")
    print("  • Environmental Impact Tracking")
    print("=" * 60)
    app.run(debug=True, host="0.0.0.0", port=5000)

SecureWipeX - Enterprise Data Wiping Platform
Starting server at http://127.0.0.1:5000
Features:
  • User Authentication (Register/Login)
  • Multiple Wipe Algorithms (NIST, DoD, GBM, etc.)
  • Certificate Generation with QR Codes
  • Real-time Analytics & Charts
  • CSV Export Functionality
  • Environmental Impact Tracking
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
